# Revenu médian par catégorie Sociopro

In [2]:
import pandas as pd
import os

file_path = "../data_brute/ircom_communes_complet_revenus_2022.xlsx"
output_folder = "../data_clean/"
output_file = os.path.join(output_folder, "ircom.csv")
xls = pd.ExcelFile(file_path)
df = pd.read_excel(xls)


In [27]:
df_clean = df.iloc[6:-3,:6]
df_clean.columns = ["Departement","Commune","Lib Commune","Tranche RFR","Nombre foyers fiscaux","RFR"]
df_clean["RFR"] = df_clean["RFR"] * 1000
df_clean[(df_clean["Departement"] == "570") & (df_clean["Commune"] == "470")]



,Departement,Commune,Lib Commune,Tranche RFR,Nombre foyers fiscaux,RFR
47653,570,470,Molring,Total,n.c.,n.c.n.c.n.c.n.c.n.c.n.c.n.c.n.c.n.c.n.c.n.c.n....


In [4]:

# Nettoyer et convertir en entier (remplace “n.c.” par NaN puis 0)
df_clean["Nombre foyers fiscaux"] = (
    pd.to_numeric(
        df_clean["Nombre foyers fiscaux"]
                 .astype(str)
                 .str.replace("\u00A0", "", regex=False)
                 .replace("n.c.", pd.NA),
        errors="coerce"
    )
    .fillna(0)
    .astype(int)
)


In [24]:
import pandas as pd
import os

# 📂 Charger le fichier Excel
file_path = "../data_brute/ircom_communes_complet_revenus_2022.xlsx"
output_folder = "../data_clean/"
output_file = os.path.join(output_folder, "ircom.csv")
xls = pd.ExcelFile(file_path)
df = pd.read_excel(xls)

# 🔹 Sélection des colonnes et nettoyage initial
df_clean = df.iloc[6:-3, :6]
df_clean.columns = ["Departement", "Commune", "Lib Commune", "Tranche RFR", "Nombre foyers fiscaux", "RFR"]
df_clean["RFR"] = df_clean["RFR"] * 1000

# 🔹 Nettoyage des types pour éviter les erreurs
for col in ["Nombre foyers fiscaux", "RFR"]:
    df_clean[col] = (
        pd.to_numeric(
            df_clean[col].astype(str).str.replace("\u00A0", "", regex=False).replace("n.c.", ""),
            errors="coerce"
        )
        .fillna(0)
    )

# 🔹 Fonction pour calculer médiane ou moyenne selon le cas
def compute_group_median(group):
    tranche = group[group["Tranche RFR"] != "Total"]
    total_row = group[group["Tranche RFR"] == "Total"]

    # Vérifier si une ligne "Total" est présente
    if not total_row.empty:
        total_row = total_row.iloc[0]
        total_foyers = total_row["Nombre foyers fiscaux"]
    else:
        total_foyers = 0

    if tranche.empty:
        # Cas où seule la ligne "Total" existe → moyenne simple
        if not total_row.empty and total_foyers != 0:
            return pd.Series({
                "Departement": total_row.get("Departement", None),
                "Commune": total_row.get("Commune", None),
                "Lib Commune": total_row.get("Lib Commune", None),
                "Nombre total de foyers fiscaux": total_foyers,
                "RFR": total_row["RFR"] / total_foyers
            })
        return pd.Series({"Departement": None, "Commune": None, "Lib Commune": None, "Nombre total de foyers fiscaux": None, "RFR": float("nan")})

    # 🔹 Sinon, interpolation de la médiane
    N = tranche["Nombre foyers fiscaux"].sum()
    half = N / 2
    cum = 0

    for _, row in tranche.iterrows():
        prev = cum
        cum += row["Nombre foyers fiscaux"]
        if cum >= half:
            if "à" in row["Tranche RFR"]:
                lo, hi = row["Tranche RFR"].split(" à ")
                lo, hi = float(lo.replace(" ", "")), float(hi.replace(" ", ""))
            else:
                lo = float(row["Tranche RFR"].split()[2].replace(" ", ""))
                hi = lo * 1.5
            return pd.Series({
                "Departement": row["Departement"],
                "Commune": row["Commune"],
                "Lib Commune": row["Lib Commune"],
                "Nombre total de foyers fiscaux": total_foyers,
                "RFR": lo + ((half - prev) / row["Nombre foyers fiscaux"]) * (hi - lo)
            })

# 🔹 Application sur tout le DataFrame avec `group_keys=False` pour éviter le DeprecationWarning
df_clean = (
    df_clean
    .sort_values(["Departement", "Commune"])
    .groupby(["Departement", "Commune"], group_keys=False)  # Correction ici
    .apply(compute_group_median)
    .reset_index(drop=True)
)

# 🔹 Afficher quelques résultats
df_clean.head()


/tmp/ipykernel_6472/965950562.py:78: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(compute_group_median)


,Departement,Commune,Lib Commune,Nombre total de foyers fiscaux,RFR
0,010,001,L'Abergement-Clémenciat,466.0,35291.096567
1,010,002,L'Abergement-de-Varey,143.0,34788.930070
2,010,004,Ambérieu-en-Bugey,9276.0,20036.511416
3,010,005,Ambérieux-en-Dombes,1073.0,26816.557692
4,010,006,Ambléon,78.0,26655.576923


In [15]:
df_clean = df_clean.dropna(how="all")


In [16]:
df_clean[df_clean.isnull().any(axis=1)]


,Departement,Commune,Lib Commune,Nombre total de foyers fiscaux,RFR
